# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1700607550000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-10-21&endtime=2023-11-20',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 10131}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.88,
  'place': '20 km NW of Challis, Idaho',
  'time': 1700436594810,
  'updated': 1700492629460,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/mb90033378',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=mb90033378&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 54,
  'net': 'mb',
  'code': '90033378',
  'ids': ',mb90033378,',
  'sources': ',mb,',
  'types': ',origin,phase-data,',
  'nst': 11,
  'dmin': 0.3677,
  'rms': 0.19,
  'gap': 143,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.9 - 20 km NW of Challis, Idaho'},
 'geometry': {'type': 'Point',
  'coordinates': [-114.389, 44.6521666666667, 8.95]},
 'id': 'mb90033378'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.88,"20 km NW of Challis, Idaho",1700436594810,1700492629460,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",mb90033378,",",mb,",",origin,phase-data,",11.0,0.367700,0.19,143.0,ml,earthquake,"M 1.9 - 20 km NW of Challis, Idaho"
1,1.19,"7 km SE of Coso Junction, CA",1700436508280,1700506956882,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39715442,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",24.0,0.004713,0.15,68.0,ml,earthquake,"M 1.2 - 7 km SE of Coso Junction, CA"
2,4.60,"210 km WNW of Panguna, Papua New Guinea",1700436500780,1700437982040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us6000lpim,",",us,",",origin,phase-data,",33.0,2.119000,0.50,119.0,mb,earthquake,"M 4.6 - 210 km WNW of Panguna, Papua New Guinea"
3,2.10,"1 km NNE of The Geysers, CA",1700436192190,1700452031118,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73964731,",",nc,",",focal-mechanism,nearby-cities,origin,phase-da...",38.0,0.011160,0.04,42.0,md,earthquake,"M 2.1 - 1 km NNE of The Geysers, CA"
4,2.60,"45 km SW of Deering, Alaska",1700436169798,1700460729040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us6000lpjb,",",us,",",origin,phase-data,",23.0,0.557000,0.56,96.0,ml,earthquake,"M 2.6 - 45 km SW of Deering, Alaska"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>